In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.worksheet import Worksheet
import os
import datetime
import re

In [2]:
df = pd.read_excel('Pricing.xlsx',sheet_name="Sheet2")
df.columns

Index(['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [3]:
df['PRODUCT SKU'] = df['PRODUCT SKU'].str.strip().str.upper()
df['PRODUCT NAME'] = df['PRODUCT NAME'].str.strip().str.upper()
df['Shop Name'] = df['Shop Name'].str.strip().str.upper()

# Append ' EBAY' to "Shop Name" where it equals 'WA INDUSTRIAL SUPPLIES'
df.loc[df['Shop Name'] == 'WA INDUSTRIAL SUPPLIES', 'Shop Name'] += ' EBAY'


df_sub = df[['BRAND', 'PRODUCT SKU', 'PRODUCT NAME', 'Shop Name', 'PRODUCT LINK',
       'Note', 'BUNDLE LINK', 'Comment']].copy()

In [4]:
# Forward fill only select columns
df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']] = df_sub[['BRAND', 'PRODUCT SKU','PRODUCT NAME']].ffill()


df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [5]:
#Example for Electroweld subset
elctro = df_sub[df_sub['Shop Name'].str.contains('electroweld', case=False, na=False, regex=True)][['Shop Name','PRODUCT LINK']]

elctro

,Shop Name,PRODUCT LINK
0,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...
22,ELECTROWELD EBAY,https://www.ebay.com.au/itm/285104945700?itmme...
25,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
35,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
...,...,...
1033,ELECTROWELD EBAY,NaN
1061,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...
1062,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275214903410?_skw=...
1092,ELECTROWELD WEBSITE,NaN


In [6]:
import httpx
from parsel import Selector
import numpy as np
import pandas as pd

def get_ebay_price(url: str) -> str:
    """
    Given an eBay product URL, fetches the page and returns the original price.
    If a timeout occurs, returns np.nan.
    """
    # First check if url is a valid, non-empty string.
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan

    # Now safe to check length (though this is redundant with url.strip() check)
    if len(url) == 0:
        return -1
    
    # establish our HTTP2 client with browser-like headers
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
        },
        http2=True,
        follow_redirects=True
    )
    
    try:
        # Fetch the webpage with an explicit timeout of 15 seconds.
        response = session.get(url, timeout=15.0)
    except httpx.ReadTimeout:
        return np.nan
    except Exception as e:
        return np.nan

    sel = Selector(response.text)
    
    # helper function to extract and strip text via CSS selectors
    css = lambda query: sel.css(query).get(default="").strip()
    
    # extract the original price
    price_original = css(".x-price-primary>span::text")
    
    return price_original

# Example usage:
if __name__ == "__main__":
    url = "https://www.ebay.com.au/itm/275880137475?itmmeta=01J3RRGZ6RGRZ1PA0NB89D3ANK&hash=item403bbcd303:g:FG4AAOSwW1lmc0i2:sc:AU_RegularParcelWithTrackingAndSignature!2190!AU!-1&itmprp=enc%3AAQAJAAAA0Jo7zG6ZrYn%2F5GTELNdvHESu%2F8%2BPfiWY9kU--0LUo18ZDC%2BkIaIZvbRp1qI9So5TpISckKJtf4oGgV7V5XLYXMhG%2FLVX497kkF4F%2BcZyu0ELdaH4TYhi3PZ%2B%2BG7xxw75rLO07cwbga6GANQK6eH5xnlQBFG02TBJBi4CCfYH8LVydeSYVPiJ8rWQKbaKABcsxdxl%2B4uJjP6Ops8YFB%2BzE7T3l62OMSCzA63O1oEmgLiiOQySiIwlAiRx%2Fu3oj4L0GrlW2ZSPf1mHsjewL9BaDsQ%3D%7Ctkp%3ABk9SR7zzw5ieZA"
    print(get_ebay_price(url))


AU $378.95


In [7]:
def get_electroweld_website_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    """
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return -1
    if not url.startswith("https://www.electroweld.com.au/product/"):
        return -1
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                          "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    try:
        response = session.get(url)
    except httpx.ReadTimeout:
        return np.nan
    except Exception as e:
        return np.nan
    sel = Selector(response.text)
    css = lambda query: sel.css(query).get(default="").strip()
    

    # Extract the price text from the page.
    price_text = css("p.w-post-elm.product_field.price span.woocommerce-Price-amount.amount bdi::text")
    # Remove a leading currency symbol if present.
    if price_text.startswith("$"):
        price_text = price_text[1:]
    return price_text

url = "https://www.electroweld.com.au/product/unimig-viper-135-multi-3-in-1-mig-tig-stick-welder-welding-torch-mma-u11005k/"
price = get_electroweld_website_price(url)
print("Product Price:", price)

Product Price: 379.00


In [8]:
def get_bilba_website_price(url: str) -> str:

    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://bilba.com.au/products"):
        return np.nan
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is extracted from a <span> element with class "price-item price-item-regular".
    """
    # Create an HTTP client with appropriate headers.
    with httpx.Client(
        headers={
            "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                           "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    ) as client:
        response = client.get(url)

    # Use parsel to parse the HTML content.
    sel = Selector(response.text)
    
    # Extract the price text from the <span> element.
    price_text = sel.css("span.price-item.price-item-regular::text").get(default="").strip()
    
    # Optionally remove any leading currency symbol, if needed.
    if price_text.startswith("$"):
        price_text = price_text[1:]
        
    return price_text
url = "https://bilba.com.au/products/unimig-razor-multi-175-welder?srsltid=AfmBOoqrcyItnYkdHvmmHMrp6Z7N-pFgOuiv_wJhyPmV19CKmUt5LIBi"
price = get_bilba_website_price(url)
print( price)

1,439.00


In [9]:
def get_gentronics_website_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in a <p> element with class="gentronics-price price".
    """
    # Basic checks for URL validity
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    
    # Check that the URL starts with the desired domain
    if not url.startswith("https://www.googleadservices.com/pagead/aclk") and not url.startswith("https://www.gentronics.com.au/"):
        return np.nan
    
    # Create an HTTP client with appropriate headers
    session = httpx.Client(
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    
    # Fetch the page
    response = session.get(url)
    sel = Selector(response.text)
    
    # Helper function for CSS selection
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text from <p class="gentronics-price price">
    price_text = css("p.gentronics-price.price::text")
    
    # Remove a leading currency symbol if present
    if price_text.startswith("$"):
        price_text = price_text[1:]
    price_text = price_text.replace("per item", "").strip()
    
    return price_text

# Example usage
if __name__ == "__main__":
    test_url = (
        "https://www.googleadservices.com/pagead/aclk?sa=L&ai=DChcSEwjZ9uvXloaLAxU2qGYCHVOQD4oYABAEGgJzbQ&ae=2&aspm=1&co=1&ase=5&gclid=Cj0KCQiAhbi8BhDIARIsAJLOlufVl2KRd0vNv4v9hFa9sC_238MQii4XHsBkOTRn3QUpKMJwPGiQne4aAoLtEALw_wcB&ohost=www.google.com&cid=CAESVeD2k-VpUGbx3aGmilsMu45ZMV1Bprr20N9Gm98bkCSwJQ8jp06PNpCmx-boJX920KleNkC6xKJ8zDKPZs-bKKEYQsYqhnX8VLcD2NqlOMrbMcc2p-A&sig=AOD64_03qiZGD-PJa-lfD8uzdS3I3IM9kQ&ctype=5&q=&ved=2ahUKEwjTsubXloaLAxXByTgGHQiRMmYQww8oAnoECAgQCg&adurl="
        # your full URL here
    )
    price = get_gentronics_website_price(test_url)
    print(price)


1199.00


In [10]:
def get_weld_com_au_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in:
    <p class="price">
        <span class="woocommerce-Price-amount amount">
            <bdi>
                <span class="woocommerce-Price-currencySymbol">$</span>1,669.00
            </bdi>
        </span>
    </p>
    """
    # Basic checks for URL validity
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return -1
    # Ensure the URL starts with the Weld product URL base.
    if not url.startswith("https://www.weld.com.au/product/"):
        return -1
    
    # Create an HTTP client with appropriate headers.
    session = httpx.Client(
        headers={
            "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        },
        follow_redirects=True
    )
    
    # Fetch the page
    response = session.get(url)
    sel = Selector(response.text)
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text using the appropriate CSS selector.
    price_text = css("p.price span.woocommerce-Price-amount.amount bdi::text")
    
    # Remove a leading currency symbol if present.
    if price_text.startswith("$"):
        price_text = price_text[1:]
    
    return price_text

# Example usage:
if __name__ == "__main__":
    url = "https://www.weld.com.au/product/unimig-razor-multi-175-bundle-pk11091/?v=6502139931c4"
    price = get_weld_com_au_price(url)
    print( price)


1,669.00


In [11]:
def get_weldconnect_price(url: str) -> str:

    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://www.weldconnect.com.au/"):
        return np.nan

    try:
        with httpx.Client(
            headers={
                "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                               "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            },
            follow_redirects=True
        ) as session:
            # Set an explicit timeout (in seconds) for the request.
            response = session.get(url, timeout=15.0)
    except httpx.ReadTimeout:
        # If a timeout occurs, return np.nan (or any other appropriate fallback)
        return np.nan

    sel = Selector(response.text)
    # Extract the price using the "content" attribute from the <div> element.
    price_text = sel.css("div.h1[itemprop='price']::attr(content)").get(default="").strip()
    
    return price_text



In [31]:
import httpx
import numpy as np
import pandas as pd
from parsel import Selector

def get_metweld_price(url: str) -> str:
    """
    Fetch the product page from the given URL and return the product price as a string.
    The price is stored in an element like:
    
    <span data-product-price-with-tax="" class="price price--withTax">$979.00</span>
    
    Returns np.nan for any failure.
    """
    # Validate URL
    if pd.isna(url) or not isinstance(url, str) or url.strip() == "":
        return np.nan
    if len(url) == 0:
        return np.nan
    if not url.startswith("https://metweld.com.au/"):
        return np.nan

    try:
        # Create an HTTP client with browser-like headers
        with httpx.Client(
            headers={
                "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                               "AppleWebKit/537.36 (KHTML, like Gecko) "
                               "Chrome/113.0.0.0 Safari/537.36"),
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            },
            follow_redirects=True
        ) as session:
            # Fetch the page with a 15-second timeout
            response = session.get(url, timeout=15.0)
    except (httpx.ReadTimeout, Exception):
        return np.nan

    sel = Selector(response.text)
    # Helper function for CSS selection
    css = lambda query: sel.css(query).get(default="").strip()
    
    # Extract the price text from the <span> element
    price_text = css("span.price.price--withTax::text")
    
    # Remove a leading currency symbol if present
    if price_text.startswith("$"):
        price_text = price_text[1:]
    
    return price_text

# Example usage:
if __name__ == "__main__":
    url = "https://metweld.com.au/viper-185-mig-tig-stick-welder/"
    price = get_metweld_price(url)
    print("Product Price:", price)


Product Price: 979.00


In [12]:
df_sub.head()

,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment
0,Unimig,U11005K,VIPER 135,ELECTROWELD WEBSITE,https://www.electroweld.com.au/product/unimig-...,NaN,NaN,NaN
1,Unimig,U11005K,VIPER 135,ELECTROWELD EBAY,https://www.ebay.com.au/itm/275880137475?itmme...,NaN,NaN,NaN
2,Unimig,U11005K,VIPER 135,HAMPDON EBAY,https://www.ebay.com.au/sch/i.html?_from=R40&_...,NaN,NaN,NaN
3,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES WEBSITE,https://www.waindustrialsupplies.net/product/v...,NaN,NaN,NaN
4,Unimig,U11005K,VIPER 135,WA INDUSTRIAL SUPPLIES EBAY,https://www.ebay.com.au/itm/186207493192?epid=...,NaN,NaN,NaN


In [13]:
# Subset dataframe to retain only EBAY shop records
df_ebay = df_sub[df_sub['Shop Name'].str.contains('EBAY', case=False, na=False, regex=True)]

In [14]:
df_electroweld_website = df_sub[df_sub['Shop Name'].str.contains('electroweld website', case=False, na=False, regex=True)]

In [15]:
df_biba_website = df_sub[
    df_sub['Shop Name'].str.contains('Bilba', case=False, na=False) &
    ~df_sub['Shop Name'].str.contains('BILBA EBAY', case=False, na=False)
]


In [16]:
df_biba_website['Price']= df_biba_website['PRODUCT LINK'].apply(get_bilba_website_price)
df_biba_website['Price_Bundle']= df_biba_website['BUNDLE LINK'].apply(get_bilba_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/735547577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biba_website['Price']= df_biba_website['PRODUCT LINK'].apply(get_bilba_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/735547577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_biba_website['Price_Bundle']= df_biba_website['BUNDLE LINK'].apply(get_bilba_website_price)


In [17]:
df_electroweld_website['Price']= df_electroweld_website['PRODUCT LINK'].apply(get_electroweld_website_price)
df_electroweld_website['Price_Bundle']= df_electroweld_website['BUNDLE LINK'].apply(get_electroweld_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2090678223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electroweld_website['Price']= df_electroweld_website['PRODUCT LINK'].apply(get_electroweld_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2090678223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_electroweld_website['Price_Bundle']= df_electroweld_website['BUNDLE LINK'].apply(get_electroweld_website_price)


In [18]:
# Apply the get_ebay_price() on entire ebad df for fetching price for all applicable url rows
df_ebay['Price']= df_ebay['PRODUCT LINK'].apply(get_ebay_price)
df_ebay['Price_Bundle']= df_ebay['BUNDLE LINK'].apply(get_ebay_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2462847340.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price']= df_ebay['PRODUCT LINK'].apply(get_ebay_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2462847340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ebay['Price_Bundle']= df_ebay['BUNDLE LINK'].apply(get_ebay_price)


In [19]:
df_gentronics = df_sub[df_sub['Shop Name'].str.contains('GENTRONICS', case=False, na=False, regex=True)]
df_gentronics['Price']= df_gentronics['PRODUCT LINK'].apply(get_gentronics_website_price)
df_gentronics['Price_Bundle']= df_gentronics['BUNDLE LINK'].apply(get_gentronics_website_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2789824565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gentronics['Price']= df_gentronics['PRODUCT LINK'].apply(get_gentronics_website_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/2789824565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gentronics['Price_Bundle']= df_gentronics['BUNDLE LINK'].apply(get_gentronics_website_price)


In [20]:
df_weldconnect = df_sub[df_sub['Shop Name'].str.contains('WELDCONNECT', case=False, na=False, regex=True)]
df_weldconnect['Price']= df_weldconnect['PRODUCT LINK'].apply(get_weldconnect_price)
df_weldconnect['Price_Bundle']= df_weldconnect['BUNDLE LINK'].apply(get_weldconnect_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/3825842770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weldconnect['Price']= df_weldconnect['PRODUCT LINK'].apply(get_weldconnect_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/3825842770.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weldconnect['Price_Bundle']= df_weldconnect['BUNDLE LINK'].apply(get_weldconnect_price)


,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment,Price,Price_Bundle
14,Unimig,U11005K,VIPER 135,WELDCONNECT,https://www.weldconnect.com.au/unimig-viper-mu...,NaN,NaN,NaN,362.73,NaN
39,Unimig,U11006K,VIPER 165,WELDCONNECT,https://www.weldconnect.com.au/unimig-viper-18...,NaN,NaN,NaN,908.19,NaN
85,Unimig,U11011,VIPER 195,WELDCONNECT,https://www.weldconnect.com.au/viper-multi-195...,NaN,NaN,NaN,1180.91,NaN
107,Unimig,U11003K,RAZOR 200 PULSE,WELDCONNECT,https://www.weldconnect.com.au/razor-200-pulse...,NaN,NaN,NaN,2453.46,NaN
126,Unimig,U11004K,RAZOR 230,WELDCONNECT,https://www.weldconnect.com.au/unimig-razor-mu...,NaN,NaN,NaN,2264.37,NaN
145,Unimig,U11007K,RAZOR MULTI 175,WELDCONNECT,https://www.weldconnect.com.au/razor-multi-175...,NaN,NaN,NaN,1308.19,NaN
173,Unimig,U11012,RAZOR MULTI 200 PFC,WELDCONNECT,https://www.weldconnect.com.au/razor-arc-200-p...,NaN,NaN,NaN,853.64,NaN
196,Unimig,U11008K,RAZOR MULTI 220,WELDCONNECT,https://www.weldconnect.com.au/razor-multi-220...,NaN,NaN,NaN,1603.64,NaN
225,Unimig,U11009K,RAZOR MULTI 250,WELDCONNECT,https://www.weldconnect.com.au/razor-multi-250...,NaN,NaN,NaN,2208.19,NaN
254,Unimig,U11010K,RAZOR COMPACT 250,WELDCONNECT,https://www.weldconnect.com.au/razor-compact-2...,NaN,NaN,NaN,2503.64,NaN


In [21]:
df_weld_com = df_sub[df_sub['Shop Name'].str.contains('WELD.COM.AU', case=False, na=False, regex=True)]
df_weld_com['Price']= df_weld_com['PRODUCT LINK'].apply(get_weld_com_au_price)
df_weld_com['Price_Bundle']= df_weld_com['BUNDLE LINK'].apply(get_weld_com_au_price)

/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/1079567426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weld_com['Price']= df_weld_com['PRODUCT LINK'].apply(get_weld_com_au_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/1079567426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weld_com['Price_Bundle']= df_weld_com['BUNDLE LINK'].apply(get_weld_com_au_price)


In [32]:
df_metro_welder_service = df_sub[df_sub['Shop Name'].str.contains('METRO WELDER SERVICE', case=False, na=False, regex=True)]
df_metro_welder_service['Price']= df_metro_welder_service['PRODUCT LINK'].apply(get_metweld_price)
df_metro_welder_service['Price_Bundle']= df_metro_welder_service['BUNDLE LINK'].apply(get_metweld_price)


/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/3775465057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metro_welder_service['Price']= df_metro_welder_service['PRODUCT LINK'].apply(get_metweld_price)
/var/folders/t1/nz7bssws0lq083n8ks9zxkmw0000gn/T/ipykernel_28056/3775465057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metro_welder_service['Price_Bundle']= df_metro_welder_service['BUNDLE LINK'].apply(get_metweld_price)


,BRAND,PRODUCT SKU,PRODUCT NAME,Shop Name,PRODUCT LINK,Note,BUNDLE LINK,Comment,Price,Price_Bundle
15,Unimig,U11005K,VIPER 135,METRO WELDER SERVICE,NaN,NaN,NaN,NaN,NaN,NaN
40,Unimig,U11006K,VIPER 165,METRO WELDER SERVICE,https://metweld.com.au/viper-multi-165-welder/,NaN,NaN,NaN,729.00,NaN
64,Unimig,KUMJRVM185,VIPER 185,METRO WELDER SERVICE,https://metweld.com.au/viper-185-mig-tig-stick...,NaN,NaN,NaN,979.00,NaN
87,Unimig,U11011,VIPER 195,METRO WELDER SERVICE,https://metweld.com.au/viper-multi-195-max-mig...,NaN,NaN,NaN,"1,249.00",NaN
108,Unimig,U11003K,RAZOR 200 PULSE,METRO WELDER SERVICE,https://metweld.com.au/razor-200-pulse-mig-tig...,NaN,NaN,NaN,"2,699.00",NaN
127,Unimig,U11004K,RAZOR 230,METRO WELDER SERVICE,https://metweld.com.au/razor-multi-230-ac-dc-m...,NaN,NaN,NaN,"2,495.00",NaN
147,Unimig,U11007K,RAZOR MULTI 175,METRO WELDER SERVICE,https://metweld.com.au/razor-multi-175-welder/,NaN,NaN,NaN,"1,355.00",NaN
198,Unimig,U11008K,RAZOR MULTI 220,METRO WELDER SERVICE,https://metweld.com.au/razor-multi-220-welder/,NaN,NaN,NaN,"1,665.00",NaN
227,Unimig,U11009K,RAZOR MULTI 250,METRO WELDER SERVICE,https://metweld.com.au/razor-multi-250-welder/,NaN,NaN,NaN,"2,289.00",NaN
256,Unimig,U11010K,RAZOR COMPACT 250,METRO WELDER SERVICE,https://metweld.com.au/razor-compact-250-welder/,NaN,NaN,NaN,"2,599.00",NaN


In [22]:
df_ebay.to_csv('Pricing_Ebay_append.csv',index=False)

In [23]:
df_electroweld_website.to_csv('Pricing_Electroweld_website_append.csv',index=False)

In [24]:
df_biba_website.to_csv('Pricing_Bilba_website_append.csv', index=False)

In [33]:
combined_df = pd.concat([df_ebay, df_electroweld_website, df_biba_website,df_gentronics,df_weld_com,df_weldconnect,df_metro_welder_service], ignore_index=True)

# Sort the combined dataframe by the "PRODUCT NAME" column
combined_df.sort_values("PRODUCT NAME", inplace=True)

In [34]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 90 to 398
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BRAND         400 non-null    object
 1   PRODUCT SKU   400 non-null    object
 2   PRODUCT NAME  400 non-null    object
 3   Shop Name     400 non-null    object
 4   PRODUCT LINK  207 non-null    object
 5   Note          3 non-null      object
 6   BUNDLE LINK   2 non-null      object
 7   Comment       1 non-null      object
 8   Price         205 non-null    object
 9   Price_Bundle  2 non-null      object
dtypes: object(10)
memory usage: 34.4+ KB


In [35]:
combined_df.to_csv('combined.csv',index=False)

In [36]:
completed_unique_shops = combined_df['Shop Name'].unique()
print(completed_unique_shops)

['HAMPDON EBAY' 'METRO WELDER SERVICE' 'WELDCONNECT' 'GENTRONICS'
 'BILBA EBAY' 'NATIONAL WELDING EBAY' 'WA INDUSTRIAL SUPPLIES EBAY'
 'BILBA' 'ELECTROWELD WEBSITE' 'ELECTROWELD EBAY' 'WELD.COM.AU'
 'WELD.COM.AU (ARC-I)']


In [37]:
unique_shops = df_sub['Shop Name'].unique()
print(unique_shops)

['ELECTROWELD WEBSITE' 'ELECTROWELD EBAY' 'HAMPDON EBAY'
 'WA INDUSTRIAL SUPPLIES WEBSITE' 'WA INDUSTRIAL SUPPLIES EBAY' 'TKD'
 'SUPERCHEAP AUTO' 'SYDNEY TOOLS' 'TRADE TOOLS' 'TOOLS WAREHOUSE'
 'TOTAL TOOLS' 'NATIONAL WELDING' 'A&S WELDING'
 "KENNEDY'S WELDING SUPPLIES" 'WELDCONNECT' 'METRO WELDER SERVICE'
 'AUSTRALIA INDUSTRIAL GROUP' 'ALPHAWELD' 'ACL INDUSTRIAL TECHNOLOGY'
 'PRIME SUPPLIES' nan 'HARE AND FORBES' 'NATIONAL WELDING EBAY'
 'BILBA EBAY' 'BILBA' 'GENTRONICS' 'WELD.COM.AU' 'VEK TOOLS'
 'HAMPDON WEBSITE' 'WELD.COM.AU (ARC-I)' 'WELDERS ONLINE']


In [38]:
not_completed = list(set(unique_shops) - set(completed_unique_shops))
print(not_completed)
print(str(len(not_completed))+"/"+str((len(unique_shops)-1)))

['HARE AND FORBES', 'VEK TOOLS', "KENNEDY'S WELDING SUPPLIES", 'ALPHAWELD', 'TOTAL TOOLS', nan, 'WELDERS ONLINE', 'TKD', 'PRIME SUPPLIES', 'TRADE TOOLS', 'ACL INDUSTRIAL TECHNOLOGY', 'HAMPDON WEBSITE', 'TOOLS WAREHOUSE', 'SYDNEY TOOLS', 'A&S WELDING', 'NATIONAL WELDING', 'WA INDUSTRIAL SUPPLIES WEBSITE', 'AUSTRALIA INDUSTRIAL GROUP', 'SUPERCHEAP AUTO']
19/30
